## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')
data = pd.read_csv('/datasets/data.csv')
columns = data.columns
print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В колонках days_employed и total_income  отсутсвует часть значений (NaN), отсутствующие значения будем заменять на 0.

### Шаг 2. Предобработка данных

In [2]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [3]:
data = data.dropna()
data = data.reset_index(drop=True)
#data = data.fillna(0)

### Вывод

В двух колонках(общий доход и стаж работы) отсутсвовало по 2174 значений. Пропуски парные(обе колонки отсутствуют в одних и тех же строках), одна из возможных причин налия пропусков - отсутствие официального стажа, а соответсвенно и дохода (т.к. эти факторы связаны между собой). Пропуски были удалены, замена пропусков на средние или срединные значения не отразит данные корректно и внесет ошибки в дальнейшем выявлениии зависимости.

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.8+ MB
None


### Вывод

Использованна команда .asstype('int'), т.к. преобразование при помощи int() не сработает здесь

### Обработка дубликатов

Приводим все данные к общему виду, необходимо : 
1) все отрицательные числа заменить на аналогичные положительные
2) привести все столбцы с не числовыми значениями перевести в нижний регистр (визуально заметно только в 1 столбце, но применим ко всем)

Приведем все столбцы с не числовыми значениями к нижнему регистру

In [6]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

Спасибо, получается перемудрил

Замена отрицательных значений на раные им по модулю

In [7]:
def remove_minus(data):
    data = abs(data)

In [8]:
columns_remove = ['days_employed', 'total_income', 'children', 'dob_years', 'education_id','family_status_id' ]
data[columns_remove].apply(remove_minus)
print(data)

       children  days_employed  dob_years            education  education_id  \
0             1          -8437         42               высшее             0   
1             1          -4024         36              среднее             1   
2             0          -5623         33              среднее             1   
3             3          -4124         32              среднее             1   
4             0         340266         53              среднее             1   
5             0           -926         27               высшее             0   
6             0          -2879         43               высшее             0   
7             0           -152         50              среднее             1   
8             2          -6929         35               высшее             0   
9             0          -2188         41              среднее             1   
10            2          -4171         36               высшее             0   
11            0           -792         4

Поиск и удаление полных дупликатов ( совпадение по всем столбцам ) и их удаление

In [9]:
#duplicatedRows = data[data.duplicated(columns)]
duplicatedRows = data.drop_duplicates().reset_index(drop
= True)
print(duplicatedRows.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.8+ MB
None


### Вывод

Полных дубликатов не было найдено.  Т.к. ни одного полного дубликата нет считаем, что каждая строка - уникальный клиент банка.( Дубликаты могли пы появиться в случае если клиент повторно обратился в банка для заема кредита)

### Лемматизация

In [10]:
lem_list = data['purpose'].tolist()
lem_list_fin = []
for element in lem_list:
    if element == 'покупка жилой недвижимости':
        lem_list_fin.append('покупка жилой')
    elif element == 'строительство жилой недвижимости':
        lem_list_fin.append('покупка жилой')
    else:
        lem_list_fin.append(element)
lem_list_fin = ', '.join(lem_list_fin)
#print(lem_list_fin)
lemmas = m.lemmatize(lem_list_fin)
print(lemmas)

['покупка', ' ', 'жилье', ', ', 'приобретение', ' ', 'автомобиль', ', ', 'покупка', ' ', 'жилье', ', ', 'дополнительный', ' ', 'образование', ', ', 'сыграть', ' ', 'свадьба', ', ', 'покупка', ' ', 'жилье', ', ', 'операция', ' ', 'с', ' ', 'жилье', ', ', 'образование', ', ', 'на', ' ', 'проведение', ' ', 'свадьба', ', ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ', ', 'покупка', ' ', 'недвижимость', ', ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ', ', 'приобретение', ' ', 'автомобиль', ', ', 'покупка', ' ', 'жилой', ', ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ', ', 'недвижимость', ', ', 'строительство', ' ', 'недвижимость', ', ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ', ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ', ', 'недвижимость', ', ', 'приобретение', ' ', 'автомобиль', ', ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ', ', 'сыграть', ' ', 'свадьба', ', ', 'операция', ' ', 'с', ' ', 'коммерче

### Вывод

При помощи функции создали список со всеми элементами столбца целей кредита, создали новый список в котором заменили цель 'покупка жилой недвижимости ' и 'строительство жилой недвижимости' на 'покупка жилой'. Смысл в принципе не изменяется (считаем все как недвижимость) а в последующих манипуляциях это вносило ошибки

### Категоризация данных

In [11]:
print(Counter(lemmas))

Counter({' ': 29122, ', ': 19350, 'покупка': 5933, 'недвижимость': 4597, 'жилье': 4035, 'автомобиль': 3897, 'образование': 3597, 'с': 2619, 'операция': 2334, 'свадьба': 2099, 'на': 2016, 'свой': 2013, 'высокий': 1243, 'получение': 1179, 'коммерческий': 1178, 'для': 1158, 'жилой': 1126, 'строительство': 1121, 'сделка': 844, 'дополнительный': 810, 'заниматься': 797, 'подержать': 771, 'сыграть': 693, 'проведение': 685, 'сдача': 588, 'семья': 570, 'собственный': 560, 'со': 559, 'ремонт': 542, 'приобретение': 419, 'профильный': 389, 'подержанный': 101, '\n': 1})


Основные цели получения кредита:  недвижимость, приобретение автомобиля, образование, свадьба. Для выявления в дальнешем статистики необходимо создать строки идентификаторы для целей кредита, наличия детей, уровня дохода. Для семейного положения идентификатор в наличии (есть супруг 0, гражданский брак 1, вдовцы 2, в разводе 3, нет супруга 4) 

In [12]:
data.loc[data['children'] != 0, 'children_id'] = 1
data.loc[data['children'] == 0, 'children_id'] = 0
data['children_id'] = data['children_id'].astype('int')
print(data['children_id'].unique())
print(data['children'].head(5))   

[1 0]
0    1
1    1
2    0
3    3
4    0
Name: children, dtype: int64


Значения индексов для children_id 1- есть дети 2 - их отсутсвие

Для индексации доходов найдем срединное значение

In [13]:
print(data['total_income'].median())

145017.0


Примем значения в диапазоне 20 тысяч от срединного как средний доход и проведем индексацию. 0 - доход ниже среднего, 1 - средний доход, 2 - высокий доход

In [14]:
data.loc[data['total_income'] < 50000 , 'total_income_id'] = 0
data.loc[(data['total_income'] >= 50000) & (data['total_income'] < 145000) , 'total_income_id'] = 1
data.loc[data['total_income'] >= 145000, 'total_income_id'] = 2
data['total_income_id'] = data['total_income_id'].astype('int')
print (data['total_income_id'].head(10))

0    2
1    1
2    2
3    2
4    2
5    2
6    2
7    1
8    1
9    1
Name: total_income_id, dtype: int64


Для начала выявления зависимостей осталось проиндексировать цель кредита (Основные цели получения кредита:  недвижимость, приобретение автомобиля, образование, свадьба.)#проблема в жилой недвижимости получаю лишние значения

In [15]:
edu = russian_stemmer.stem('образование') #(0)
auto = russian_stemmer.stem('автомобиль') #(1)
marry = russian_stemmer.stem('свадьба') #(2)
hom1 = russian_stemmer.stem('недвижимость') #(3)
hom2 = russian_stemmer.stem('жилье') #(3)
data_list = list()
for query in lemmas:  
        for word in query.split():
            stemmed_word = russian_stemmer.stem(word)
            if stemmed_word == edu:
                data_list += '0'  
            elif stemmed_word == auto:
                data_list += '1'
            elif stemmed_word == marry:
                data_list += '2'
            elif (stemmed_word == hom1) or (stemmed_word == hom2) :
                data_list += '3'
print(len(data_list))

19351


Добавляем столбец с индексами целей кредита в изначальный фрейм

In [16]:
df = pd.DataFrame(data_list,columns=['purpose_id'])
data['purpose_id'] = df['purpose_id']
data['purpose_id'] = data['purpose_id'].astype('int')
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 15 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
children_id         19351 non-null int64
total_income_id     19351 non-null int64
purpose_id          19351 non-null int64
dtypes: int64(10), object(5)
memory usage: 2.2+ MB
None


### Вывод

Все интересующие нас данные были разбиты по категориям :
1) по цели кредита 0 - на обучение, 1 -на автомобиль, 2 - свадьба, 3 - на недвижимость
2) по достатку относительного срединного значения 0 - ниже среднего, 1 - средний достаток, 2 - высокий достаток
3) по наличию детей 0 - нет детей, 1 - есть дети (вне зависимости от количества)
4) задолженность 1 - имел 0- не имел

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
with_children = data.loc[data['children_id'] == 1]['children_id'].count()
no_children = data.loc[data['children_id'] == 0]['children_id'].count()
very_happy = data.loc[(data['debt'] == 0) & (data['children_id'] == 1)]['children_id'].count()
happy = data.loc[(data['debt'] == 0) & (data['children_id'] == 0)]['children_id'].count()
#not_good = data.loc[(data['debt'] == 1) & (data['children_id'] == 1)]['children_id'].count()
#bad = data.loc[(data['debt'] == 1) & (data['children_id'] == 0)]['children_id'].count()
returns_children = very_happy / with_children
returns = happy / no_children

In [18]:
print('Клиенты с детьми: ',with_children)
print('Клиенты без детей: ',no_children)
#print('Клиенты с детьми и без задолженности: ', very_happy)
#print('Клиенты без детей и без задолженности: ', happy)
#print('Клиенты с детьми и с задолженностью: ', not_good)
#print('Клиенты без детей и с задолженнотью: ', bad)
print('Процент клиентов с детьми и без задолженности: {:.1%}'.format(returns_children))
print('Процент клиентов без детей и без задолженности: {:.1%}'.format(returns))

Клиенты с детьми:  6641
Клиенты без детей:  12710
Процент клиентов с детьми и без задолженности: 90.7%
Процент клиентов без детей и без задолженности: 92.5%


### Вывод

Было взято соотношение людей без задолженностей и с ним при наличии/отсутсвии детей. В обоих случаях процент возврата кредитов в срок составил около 90%, т.е. нет явно выраженого преобладания такого фактора как наличие детей при возврате кредита

In [19]:
data['debt'].mean()

0.0811844349129244

In [20]:
len(data)

19351

In [21]:
data.pivot_table(values = 'debt', index = ['children_id'], aggfunc = ['count','mean'])

,count,mean
,debt,debt
children_id,,
0,12710,0.074902
1,6641,0.093209


In [22]:
(data.groupby(['children_id'])['debt']
 .agg(['count', 'mean'])
 .style
 .format({
     'mean':'{:,.1%}'.format
 }))

,count,mean
children_id,,
0,12710,7.5%
1,6641,9.3%


В столбце mean получили процент клиентов имеющие задолженности, 7,5 % и 9,3 % для клиентов без и с детьми соответсвенно. (Такую сводную таблицу имелось ввиду? Как в через pivot форматировать в % пока не смог разобраться если честно)

### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
hight_income = data.loc[data['total_income_id'] == 2]['total_income_id'].count()
middle_income = data.loc[data['total_income_id'] == 1]['total_income_id'].count()
low_income = data.loc[data['total_income_id'] == 0]['total_income_id'].count()
hight_happy = data.loc[(data['debt'] == 0) & (data['total_income_id'] == 2)]['total_income_id'].count()
middle_happy = data.loc[(data['debt'] == 0) & (data['total_income_id'] == 1)]['total_income_id'].count()
low_happy = data.loc[(data['debt'] == 0) & (data['total_income_id'] == 0)]['total_income_id'].count()
hight_result = hight_happy / hight_income
middle_result = middle_happy / middle_income
low_result = low_happy / low_income

In [24]:
print('Клиенты с высоким доходом: ', hight_income)
print('Клиенты со средним доходом: ', middle_income)
print('Клиенты с низним доходом: ', low_income)
print('Процент клиентов без задолженности и высоким доходом: {:.1%}'.format(hight_result))
print('Процент клиентов без задолженности и средним доходом: {:.1%}'.format(middle_result))
print('Процент клиентов без задолженности и низким доходом: {:.1%}'.format(low_result))

Клиенты с высоким доходом:  9677
Клиенты со средним доходом:  9302
Клиенты с низним доходом:  372
Процент клиентов без задолженности и высоким доходом: 92.1%
Процент клиентов без задолженности и средним доходом: 91.6%
Процент клиентов без задолженности и низким доходом: 93.8%


### Вывод

И вновь нет явного преобладания ни в одной из доходных категорий, одинакого  возвращают клиенты не зависимо от дохода  

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
all_auto = data.loc[data['purpose_id'] == 1]['purpose_id'].count()
all_home = data.loc[data['purpose_id'] == 3]['purpose_id'].count() 
all_edu  = data.loc[data['purpose_id'] == 0]['purpose_id'].count() 
all_marry = data.loc[data['purpose_id'] == 2]['purpose_id'].count()
happy_auto =  data.loc[(data['debt'] == 0) & (data['purpose_id'] == 1)]['purpose_id'].count()
happy_home = data.loc[(data['debt'] == 0) & (data['purpose_id'] == 3)]['purpose_id'].count()
happy_edu =  data.loc[(data['debt'] == 0) & (data['purpose_id'] == 0)]['purpose_id'].count()
happy_marry = data.loc[(data['debt'] == 0) & (data['purpose_id'] == 2)]['purpose_id'].count() 
auto_result = happy_auto / all_auto
home_result = happy_home / all_home
edu_result = happy_edu / all_edu
marry_result = happy_marry / all_marry 

In [26]:
print('Клиенты с кредитом на автомобиль: ', all_auto)
print('Клиенты с кредитом на недвижимость: ', all_home)
print('Клиенты с кредитом на образование: ', all_edu)
print('Клиенты с кредитом на свадьбу: ', all_marry)
print('Процент клиентов без задолженностью с кредитом на автомобиль: {:.1%}'.format(auto_result))
print('Процент клиентов без задолженностью с кредитом на недвижимость: {:.1%}'.format(home_result))
print('Процент клиентов без задолженностью с кредитом на образование: {:.1%}'.format(edu_result))
print('Процент клиентов без задолженностью с кредитом на свадьбу: {:.1%}'.format(marry_result))

Клиенты с кредитом на автомобиль:  3897
Клиенты с кредитом на недвижимость:  9758
Клиенты с кредитом на образование:  3597
Клиенты с кредитом на свадьбу:  2099
Процент клиентов без задолженностью с кредитом на автомобиль: 90.6%
Процент клиентов без задолженностью с кредитом на недвижимость: 92.7%
Процент клиентов без задолженностью с кредитом на образование: 90.8%
Процент клиентов без задолженностью с кредитом на свадьбу: 92.5%


### Вывод

Сильного разрыва между возвратами не наблюдается. наименьший процент возвратов приходиться на автомобильные и кредиты взятые для образовательных целей 

### Шаг 4. Общий вывод

Были проведены обработка данных и посчитана статистика наличия задолженностей в зависимости от различных факторов. Сильно выделяющихся категорий не было обнаружено, скорее всего т.к. при выдаче заема уже учитываются риски не возврата более чем по 1 отдельно взятому фактору